In [1]:
import numpy as np
import fieldmath

In [194]:
def bits_to_number(binary):
    number = 0
    for b in binary:
        number = (2 * number) + b
    return number

def number_to_padbase(n, k):
    if n == 0:
        return [0]*k
    digits = []
    while n:
        digits.append(int(n % 2))
        n //= 2
    nopad = digits[::-1]
    return [0]*(k-len(nopad))+nopad

def tensor(matrices):
    out = matrices.pop(0)
    while matrices:
        out = np.kron(out, matrices.pop(0))
    return out

In [195]:
a = [np.array([1,0]),np.array([0,1])]

In [196]:
a.pop(0)

array([1, 0])

In [197]:
tensor([np.array([1,0]),np.array([0,1])])

array([0, 1, 0, 0])

In [198]:
n = 5
A = [np.random.randint(2) for dummy in range(np.power(n,2))]
A = np.reshape(A, [n,n])
Ddict = {(0,0):0,(0,1):2,(1,0):4,(1,1):6}
Jdict = {0:0, 1:4}
ketbit = {0:np.array([1,0]),1:np.array([0,1])}

In [199]:
n = 3
A = np.array([[1,0,1],[1,1,0],[0,1,1]])

In [200]:
len(A)

3

In [359]:
def row_reduce_gf2(A,n):
    n = len(A)
    B = fieldmath.Matrix(n,n,fieldmath.PrimeField(2))

    for i in range(n):
        for j in range(n):
            B.set(i,j,int(A[i,j]))

    B.reduced_row_echelon_form()

    for i in range(n):
        for j in range(n):
            A[i,j]=B.get(i,j)
            
    return A[~np.all(A == 0, axis=1)]

In [202]:
row_reduce_gf2(A)

array([[1, 0, 1],
       [0, 1, 1]])

In [360]:
def convert_Dbits(x):
    assert len(x)%2 == 0
    return [Ddict[(x[2*i],x[2*i+1])] for i in range(int(len(x)/2))]

def convert_Jbits(x,n):
    assert len(x) == n*(n-1)/2
    x = [Jdict[x[i]] for i in range(len(x))]
    tri = np.zeros((n, n))
    tri[np.triu_indices(n, 1)] = x
    sym = tri + np.transpose(tri)
    return sym

In [361]:
def bits_to_stab(bitarray):
    bitarray = np.array(bitarray)
    assert len(bitarray) ==  3/2*np.power(n,2)+5/2*n+3
    h = bitarray[:n]
    A = bitarray[n:n+np.power(n,2)]
    Q = bitarray[n+np.power(n,2):n+np.power(n,2)+3]
    D = bitarray[n+np.power(n,2)+3:n+np.power(n,2)+3+2*n]
    J = bitarray[n+np.power(n,2)+3+2*n:len(bitarray)]
    
    A = np.reshape(A,[n,n])
    A = row_reduce_gf2(A)
    k = len(A)
    Q = bits_to_number(Q)
    D = convert_Dbits(D[:2*k])
    print(k)
    Jmatrix = convert_Jbits(J)[:k,:k]
    state = 0 
    for vecx_number in range(np.power(2,k)):
        vecx = number_to_padbase(vecx_number,k)
        qx = Q + np.dot(vecx,D) + np.dot(vecx,Jmatrix.dot(vecx))
        x = h + sum([A[i]*vecx[i] for i in range(k)])
        x = np.mod(x,2)
        ketx = tensor([ketbit[x[i]] for i in range(n)])
        state = state+np.exp(1j*np.pi/4*qx)*ketx
        
    state = state/np.power(2,k/2)
    
    return state

In [365]:
# bitarray = [np.random.randint(2) for dummy in range(int(n))]+[0]*np.power(n,2)+[np.random.randint(2) for dummy in range(int(3+2*n+n*(n-1)/2))]
n = 6
m = 3/2*np.power(n,2)+5/2*n+3
bitarray = [np.random.randint(2) for dummy in range(int(m))]
# bits_to_stab(bitarray)

In [366]:
a[:1]

[array([0, 1])]

In [367]:
tri = np.zeros((5, 5))
tri[np.triu_indices(5, 1)] = np.array([1,2,3,4,5,6,7,8,9,10])
tri

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  0.,  5.,  6.,  7.],
       [ 0.,  0.,  0.,  8.,  9.],
       [ 0.,  0.,  0.,  0., 10.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [397]:
import copy

In [433]:
def bits_to_stab(bits,n,chi):
    bits = np.array(bits)
    basis = []
    m = 3/2*np.power(n,2)+5/2*n+3
    assert m%1 == 0 
    m = int(m)
    assert len(bits) == chi*m
    for i in range(chi):
        bitarray = bits[i*m:(i+1)*m]
        assert len(bitarray) ==  m
        h = bitarray[:n]
        A = bitarray[n:n+np.power(n,2)]
        Q = bitarray[n+np.power(n,2):n+np.power(n,2)+3]
        D = bitarray[n+np.power(n,2)+3:n+np.power(n,2)+3+2*n]
        J = bitarray[n+np.power(n,2)+3+2*n:len(bitarray)]
        
        A = np.reshape(A,[n,n])
        A = row_reduce_gf2(A,n)
        k = len(A)
        Q = bits_to_number(Q)
        D = convert_Dbits(D[:2*k])

        Jmatrix = convert_Jbits(J,n)[:k,:k]
        state = 0 
        for vecx_number in range(np.power(2,k)):
            vecx = number_to_padbase(vecx_number,k)
            qx = Q + np.dot(vecx,D) + np.dot(vecx,Jmatrix.dot(vecx))
            x = h + sum([A[i]*vecx[i] for i in range(k)])
            x = np.mod(x,2)
            ketx = tensor([ketbit[x[i]] for i in range(n)])
            state = state+np.exp(1j*np.pi/4*qx)*ketx
            
        state = state/np.power(2,k/2)
        basis.append(state)
    assert len(bits)
    return tuple(basis)

In [434]:
a = [0,1,2]

In [435]:
a[2]

2

In [436]:
len(a)

3

In [437]:
n_qubits = 2
chi = 2
m = 3/2*np.power(n_qubits,2)+5/2*n_qubits+3
x = [np.random.randint(2) for dummy in range(int(m*chi))]

In [438]:
bits_to_stab(x,n_qubits,chi)

(array([ 0.35355339-0.35355339j,  0.35355339+0.35355339j,
         0.35355339+0.35355339j, -0.35355339+0.35355339j]),
 array([0.70710678+0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j]))

In [ ]:
# okay, the assert helped big time!

In [439]:
n = 2
x = [0,1]
[ketbit[x[i]] for i in range(n)]

[array([1, 0]), array([0, 1])]